In [1]:
import spacy
import json
import pandas as pd
import requests 
import enchant
from application import *
from application.models import Metadata, Work, Author

In [2]:
import pandas as pd
all_rows = pd.read_csv("metadata.csv")

In [24]:
urls = []
for i in all_rows['nyt_id']:
    row = Metadata().query.filter(Metadata.nyt_id == i).one_or_none()
    urls.append(row.nyt_pdf_endpoint)                              

In [27]:
all_rows['nyt_pdf_endpoint'] = urls
all_rows.to_csv("/Users/matthewlavin/nyt-reviews-1905-gender/metadata.csv")

In [3]:
female_rows = all_rows.loc[all_rows['assumed_gender'] == 'f']
male_rows = all_rows.loc[all_rows['assumed_gender'] == 'm']

print(len(male_rows), len(female_rows))

(703, 159)


In [4]:
nlp = spacy.load('en')

In [56]:
words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words')
stoplist1 = words.text.split("\r\n")

from nltk.corpus import stopwords
stoplist2 = set(stopwords.words('english'))

stoplist1.extend(stoplist2)

fullstops = list(set(stoplist1))
fullstops = [i for i in fullstops if i !='']

def remove_stops(stoplist, wordlist):
    result = []
    for i in wordlist:
        if i not in stoplist:
                result.append(i)
    return result

def spellcheck(wordlist):
    result = []
    d = enchant.Dict("en_US")
    for i in wordlist:
        if d.check(i) or d.check(i.capitalize()):
            result.append(i)
         
    return result

def get_term_tree(list_of_texts, term, nyt_ids_list):
    term_list =[]
    #normalize ocr errors
    for h,i in enumerate(list_of_texts):
        #lowercase all
        ocr_lower = i.lower()
        #tokenize, remove punctuation and numbers, remove tabs, newlines, etc.
        ocr_cleaner = ocr_lower.replace("\n", " ").replace("\t", " ")
        doc = nlp(ocr_cleaner)
        ocr_tokens = []
        #add to term list any lemma or token that matches term, but not both if they are the same
        for token in doc:
            ocr_tokens.append(unicode(token))
        for z, token in enumerate(doc):
            if token.lemma_ == term or unicode(token) == term:
                context_list = []
                if z >= 6:
                    context_list.extend(ocr_tokens[z-6:z-1])
                else: 
                    context_list.extend(ocr_tokens[0:z-1])
                
                try:
                    context_list.extend(ocr_tokens[z:z+7])
                except:
                    context_list.extend(ocr_tokens[z:[len(ocr_tokens)]])
                result = {"lemma": token.lemma_, "token": unicode(token), "pos": token.pos_, "nyt_id": nyt_ids_list[h], 
                          "context": context_list}
                term_list.append(result)
    return term_list
            
def clean_text(list_of_texts):
    fully_cleaned =[]
    #normalize ocr errors
    for i in list_of_texts:
        #lowercase all
        ocr_lower = i.lower()
        #tokenize, remove punctuation and numbers, remove tabs, newlines, etc.
        ocr_cleaner = ocr_lower.replace("\n", " ").replace("\t", " ")
        doc = nlp(ocr_cleaner)
        ocr_tokens = []
        for token in doc:
            
            if token.lemma_ == u'-PRON-' or token.lemma_.isupper():
                ocr_tokens.append(unicode(token))
            else:
                ocr_tokens.append(token.lemma_)
        #ocr_tokens = ocr_cleaner.split(" ")
        
        no_numbers_or_punct = []
        for token in ocr_tokens:
            if token.isalpha():
                no_numbers_or_punct.append(token)
            else:
                
                new_token = ""
                for letter in token:
                    if letter.isalpha():
                        new_token += letter
                if new_token != "":
                    no_numbers_or_punct.append(new_token)  
        
        
        spellchecked = spellcheck(no_numbers_or_punct)
        fully_cleaned.append(spellchecked)
    return fully_cleaned

print(fullstops)

#this list of gender terms was generated iteratively by running the logistic regression with all terms, 
#seeing what correlated the most with gender, and removing words that seemed to have direct gender info in them

gender_terms = ["mr", "he", "his", "him", "himself", "man", "men", "boy", "boys", "manly", "masculine", "boyish", "father", \
                "brother", "girls", "men", "women", "sisters", "daughters", "brothers", "sons", "wife", "husband", "niece",\
                "uncle", "nephew", "dad", "grandfather", "son", "mrs", "miss", "her", "hers", "she", "herself", "woman",\
                "girl", "nieces", "nephews", "fer", "mme", "mlle", \
                "lady", "womanly", "girlish", "girly", "mother", "daughter", "aunt", "niece" "grandmother", "mom", "sister" ]

from nltk.corpus import names 
male = [o.lower() for o in names.words('male.txt')]
female = [o.lower() for o in names.words('female.txt')]

fullstops_and_pronouns = []

for u in [fullstops, gender_terms]:
    for i in u:
        fullstops_and_pronouns.append(unicode(i))

fullstops_and_pronouns = list(set(fullstops_and_pronouns))

[u'all', u'whoever', u'four', u'go', u'mill', u'seemed', u'whose', u'to', u'under', u'very', u'every', u'yourselves', u'did', u'the', u'ten', u'further', u'even', u'what', u'weren', u'above', u'mustn', u'ever', u'thin', u'hasn', u'full', u'never', u'here', u'shouldn', u'others', u'alone', u'along', u'fifteen', u'wherever', u'amount', u'via', u'yourself', u'from', u'would', u'two', u'next', u'few', u'call', u'therefore', u'themselves', u'thru', u'until', u'more', u'becomes', u'hereby', u'herein', u'everywhere', u'must', u'me', u'none', u'ma', u'this', u'anywhere', u'nine', u'can', u'theirs', u'my', u'give', u'something', u'rather', u'six', u'how', u'needn', u'haven', u'may', u'after', u'hereupon', u'such', u'a', u'third', u'whenever', u'so', u'indeed', u'over', u'move', u'through', u'fify', u'still', u'its', u'before', u'thence', u'somewhere', u'll', u'ours', u'might', u'wouldn', u'them', u'someone', u'thereby', u'they', u'not', u'now', u'nor', u'name', u'always', u'didn', u'whither', u

In [29]:
from string import ascii_lowercase
fullstops_pronouns_and_names = []

for u in [fullstops_and_pronouns, male, female]:
    for i in u:
        fullstops_pronouns_and_names.append(unicode(i))

fullstops_pronouns_and_names.append(unicode("thoma"))

for ltr in ascii_lowercase:
    fullstops_pronouns_and_names.append(unicode(ltr))

fullstops_pronouns_and_names = list(set(fullstops_pronouns_and_names))

print(len(fullstops), len(fullstops_and_pronouns), len(fullstops_pronouns_and_names))

(353, 396, 7982)


In [30]:
ocr_list_male = []
male_nyt_ids = []
ocr_list_female = []
female_nyt_ids = []

In [31]:
for i in female_rows.iterrows():
    row = Metadata().query.filter(Metadata.id == int(i[1][0])).one_or_none()
    ocr_list_female.append(row.ocr_transcription)
    female_nyt_ids.append(row.nyt_id)

In [32]:
for i in male_rows.iterrows():
    row = Metadata().query.filter(Metadata.id == int(i[1][0])).one_or_none()
    ocr_list_male.append(row.ocr_transcription)
    male_nyt_ids.append(row.nyt_id)

In [ ]:
tree_for_volume = get_term_tree(ocr_list_male, "volume", male_nyt_ids)

In [33]:
female_nyt_ids

[u'4fc047fa45c1498b0d22001a',
 u'4fc03b9345c1498b0d1e87c6',
 u'4fc03b9345c1498b0d1e87bd',
 u'4fc03b9345c1498b0d1e87cb',
 u'4fc054cf45c1498b0d258c95',
 u'4fc0528045c1498b0d24e578',
 u'4fc0532a45c1498b0d250fe9',
 u'4fc0520845c1498b0d24b887',
 u'4fc045fe45c1498b0d216556',
 u'4fc0471145c1498b0d21b379',
 u'4fc045fe45c1498b0d2165e8',
 u'4fc0471245c1498b0d21b41e',
 u'4fc045fe45c1498b0d21664f',
 u'4fc0466845c1498b0d217f33',
 u'4fc051cb45c1498b0d24a71e',
 u'4fc052b245c1498b0d24eacc',
 u'4fc03a7c45c1498b0d1e2d1b',
 u'4fc0466845c1498b0d217f48',
 u'4fc0451845c1498b0d2122a4',
 u'4fc0466845c1498b0d217fe1',
 u'4fc052b245c1498b0d24eb40',
 u'4fc051cb45c1498b0d24a792',
 u'4fc0466945c1498b0d218089',
 u'4fc0451845c1498b0d2122c5',
 u'4fc0466945c1498b0d21807d',
 u'4fc051cb45c1498b0d24a7da',
 u'4fc03a7d45c1498b0d1e2dab',
 u'4fc0466945c1498b0d218080',
 u'4fc03a7d45c1498b0d1e2daf',
 u'4fc051cb45c1498b0d24a7ed',
 u'4fc0478545c1498b0d21d6e8',
 u'4fc043f045c1498b0d20cfa3',
 u'4fc0478545c1498b0d21d6f5',
 u'4fc039f

In [18]:
tree_for_garden = get_term_tree(ocr_list_female, "garden", female_nyt_ids)

In [ ]:
tree_for_gardener = get_term_tree(ocr_list_female, "gardener", female_nyt_ids)

In [19]:
len(list(set([u['nyt_id'] for u in tree_for_garden])))

17

In [20]:
pd.DataFrame(tree_for_garden)

,context,lemma,nyt_id,pos,token
0,"[their, ., houses, or, digs, gardens, than, to...",garden,4fc045fe45c1498b0d216556,NOUN,gardens
1,"[its, first, chapter, entitled, "", garden, of,...",garden,4fc045fe45c1498b0d2165e8,NOUN,garden
2,"[pluce, been, converted, into, a, garden, ?, t...",garden,4fc0466945c1498b0d218080,NOUN,garden
3,"[in, a, qiilet, corner, .of, garden, in, the, ...",garden,4fc03a7d45c1498b0d1e2e2c,NOUN,garden
4,"[holds, the, key, to, some, garden, of, the, i...",garden,4fc0478645c1498b0d21d8dd,NOUN,garden
5,"[the, "", littlt, palace, of, garden, ,, ”, the...",garden,4fc0478645c1498b0d21d8dd,NOUN,garden
6,"[charms, of, the, apple, in, garden, of, eden,...",garden,4fc043f145c1498b0d20d151,NOUN,garden
7,"[house, alone, ,, working, in, garden, ,, dugt...",garden,4fc043f145c1498b0d20d151,NOUN,garden
8,"[﻿thk, garden, ., by, tvllla, slbort, cather, .]",garden,4fc045fe45c1498b0d216692,NOUN,garden
9,"[fairy, palace, ., with, a, garden, ;, •, •, *...",garden,4fc045fe45c1498b0d216692,NOUN,garden


In [ ]:
from nltk.corpus import wordnet as wn
for a in wn.synsets('garden'):
    print(a.definition())


In [34]:
ocr_cleaned_male = clean_text(ocr_list_male)
ocr_cleaned_female = clean_text(ocr_list_female)

In [35]:
from collections import Counter 
ocr_counters_male = [Counter(i) for i in ocr_cleaned_male]
ocr_counters_female = [Counter(i) for i in ocr_cleaned_female]
ocr_counters_all = []
for i in ocr_counters_male:
    ocr_counters_all.append(i)
for i in ocr_counters_female:
    ocr_counters_all.append(i)
nyt_ids_all = []
for i in male_nyt_ids:
    nyt_ids_all.append(i)
for i in female_nyt_ids:
    nyt_ids_all.append(i)

In [53]:
for h,i in enumerate(nyt_ids_all):
    filename = "".join(["/Users/matthewlavin/nyt-reviews-1905-gender/lemma-data/", i, ".csv"])
    df = pd.DataFrame.from_records(ocr_counters_all[h].items(), columns=['lemma', 'count']).sort_values(by="count", ascending=False)
    df.to_csv(filename, index=False)

In [ ]:
import pickle
with open('pickled_data/ocr_counters_all.pickle', 'wb') as handle:
    pickle.dump(ocr_counters_all, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickled_data/nyt_ids_all.pickle', 'wb') as handle3:
    pickle.dump(nyt_ids_all, handle3, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('pickled_data/fullstops.pickle', 'wb') as handle2:
    pickle.dump(fullstops, handle2, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickled_data/fullstops_and_pronouns.pickle', 'wb') as handle4:
    pickle.dump(fullstops_and_pronouns, handle4, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickled_data/fullstops_pronouns_and_names.pickle', 'wb') as handle5:
    pickle.dump(fullstops_pronouns_and_names, handle5, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
nrc_rows = pd.read_csv("lexicons/NRC-emotion-lexicon.txt", sep="\t", header=None, names=["term", "valence", "score"])
nrc_rows_positive = nrc_rows.loc[(nrc_rows['score'] == 1) & (nrc_rows['valence'] == 'positive')]
nrc_rows_negative = nrc_rows.loc[(nrc_rows['score'] == 1) & (nrc_rows['valence'] == 'negative')]

pos_terms = list(nrc_rows_positive['term'])
neg_terms = list(nrc_rows_negative['term'])

def remove_dupes(source, target):
    unique = []
    for s in source: 
        if s not in target:
            unique.append(s)
    return unique

pos_terms_unique = remove_dupes(pos_terms, neg_terms)
neg_terms_unique = remove_dupes(neg_terms, pos_terms)

with open('pickled_data/nrc_positive.pickle', 'wb') as handle6:
    pickle.dump(pos_terms_unique, handle6, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickled_data/nrc_negative.pickle', 'wb') as handle7:
    pickle.dump(neg_terms_unique, handle7, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(list(nrc_rows_negative['term']) + list(nrc_rows_positive['term']))

In [ ]:
"he" == unicode("he")

In [59]:
d = enchant.Dict("en_US")
d.check("Thoma")

True